In [15]:
import pandas as pd


def generate_report(sales_file: str, products_file: str, regions_file: str, n: int = 3) -> pd.DataFrame:
    """
    Generate a sales report with best region, best category, and top N products.

    Args:
        sales_file (str): Path to sales CSV with columns [date, product_id, revenue].
        products_file (str): Path to products CSV with columns [product_id, product_name, category].
        regions_file (str): Path to regions CSV with columns [date, region].
        n (int): Number of top products to include.

    Returns:
        pd.DataFrame: Final report with summary rows.
    """
    # Load data
    sales = pd.read_csv(sales_file)
    products = pd.read_csv(products_file)
    regions = pd.read_csv(regions_file)

    # Merge
    merged = sales.merge(products, on="product_id", how="left")
    full_data = merged.merge(regions, on="date", how="left")

    # Best region
    best_region = (
        full_data.groupby("region")["revenue"]
        .sum()
        .nlargest(1)
        .reset_index()
    )
    best_region["report_type"] = "best region"

    # Best category
    best_category = (
        full_data.groupby("category")["revenue"]
        .sum()
        .nlargest(1)
        .reset_index()
    )
    best_category["report_type"] = "best category"

    # Top N products
    top_products = (
        full_data.groupby("product_name")["revenue"]
        .sum()
        .nlargest(n)
        .reset_index()
    )
    top_products["report_type"] = f"best {n} products"

    # Combine
    report = pd.concat([best_region, best_category, top_products], ignore_index=True)

    # Replace NaN with placeholder for clarity
    report = report.fillna("N/A")

    # Save
    report.to_csv("report13.csv", index=False)
    print("Report saved to report13.csv")

    return report


# Demo
report_df = generate_report("sales13.csv", "products13.csv", "regions13.csv", n=3)
print(report_df)


Report saved to report13.csv
  region  revenue      report_type     category product_name
0   West      345      best region          N/A          N/A
1    N/A      760    best category  Electronics          N/A
2    N/A      390  best 3 products          N/A       Laptop
3    N/A      280  best 3 products          N/A     Backpack
4    N/A      230  best 3 products          N/A       Tablet
